# Importing and Inspecting the Data

In [ ]:
# import depedencies
import requests as req
import pandas as pd
import datetime as dt
import matplotlib.pyplot as plt
import json

In [ ]:
url_covid = 'https://api.covidtracking.com/v1/states/daily.json'
# url_usa_population = 'https://datausa.io/api/data?drilldowns=Nation&measures=Population&year=latest'
url_state_population = 'https://datausa.io/api/data?drilldowns=State&measures=Population&year=latest'

In [ ]:
# retrieving API and storing data in variables
covid_js = req.get(url_covid).json()
# usa_pop_js = req.get(url_usa_population).json()
state_pop_js = req.get(url_state_population).json()
print(json.dumps(state_pop_js['data'], indent=4, sort_keys=True))

In [ ]:
# converting the data to a dataframe for data manipulation and cleaning
covid_all = pd.DataFrame(covid_js)
# usa_pop_df = pd.DataFrame(usa_pop_js['data'])
state_pop_df = pd.DataFrame(state_pop_js['data'])
display(covid_all.head(), state_pop_df.head())

In [ ]:
##### Emilio's Code Start Here #####

In [ ]:
# state_pop_df['Population'].sum()

In [ ]:
state_pop_df['State'].nunique()

In [ ]:
state_pop_df['State'].unique()

In [ ]:
# inspecting data
display(covid_all.shape, state_pop_df.shape)

In [ ]:
# inspecting data
covid_all.describe()

In [ ]:
# # inspecting data: fields (also by reviewing API documentation)
covid_all.columns

In [ ]:
# # inspecting data: data types
covid_all.info()

# Data Cleaning
## covid_df
#### Removing the following columns as they don't return required data, as well as deprecated fields reported in the documentation:

In [ ]:
# list of fields: all, not-required, and deprecated fields
all_fields = ['date', 'state', 'positive', 'probableCases', 'negative', 'pending',
       'totalTestResults', 'hospitalizedCurrently', 'hospitalizedCumulative',
       'inIcuCurrently', 'inIcuCumulative', 'onVentilatorCurrently',
       'onVentilatorCumulative', 'recovered', 'dataQualityGrade',
       'lastUpdateEt', 'dateModified', 'checkTimeEt', 'death', 'hospitalized',
       'dateChecked', 'totalTestsViral', 'positiveTestsViral',
       'negativeTestsViral', 'positiveCasesViral', 'deathConfirmed',
       'deathProbable', 'totalTestEncountersViral', 'totalTestsPeopleViral',
       'totalTestsAntibody', 'positiveTestsAntibody', 'negativeTestsAntibody',
       'totalTestsPeopleAntibody', 'positiveTestsPeopleAntibody',
       'negativeTestsPeopleAntibody', 'totalTestsPeopleAntigen',
       'positiveTestsPeopleAntigen', 'totalTestsAntigen',
       'positiveTestsAntigen', 'fips', 'positiveIncrease', 'negativeIncrease',
       'total', 'totalTestResultsSource', 'totalTestResultsIncrease', 'posNeg',
       'deathIncrease', 'hospitalizedIncrease', 'hash', 'commercialScore',
       'negativeRegularScore', 'negativeScore', 'positiveScore', 'score',
       'grade']
deprecated_fields = ['checkTimeEt', 'commercialScore', 'dateChecked', 'dateModified', 
                     'grade', 'hash', 'hospitalized', 'negativeIncrease', 
                     'negativeRegularScore', 'negativeScore', 'posNeg', 'positiveScore', 
                     'score', 'total', 'totalTestResultsSource']
non_required_fields = ['deathConfirmed', 'deathProbable', 'lastUpdateEt', 
                       'totalTestsViral', 'positiveTestsViral', 'negativeTestsViral', 
                       'positiveCasesViral', 'probableCases', 'negative', 'recovered',
                       'pending', 'totalTestEncountersViral', 'totalTestsPeopleViral', 
                       'totalTestsAntibody', 'positiveTestsAntibody', 'negativeTestsAntibody',
                       'totalTestsPeopleAntibody', 'positiveTestsPeopleAntibody',
                       'negativeTestsPeopleAntibody', 'totalTestsPeopleAntigen', 
                       'positiveTestsPeopleAntigen','totalTestsAntigen', 'positiveTestsAntigen', 
                       'totalTestResultsIncrease', 'totalTestResults']

In [ ]:
# removing deprecated and non-required fields to obtain the filtered list
filtered_fields = []
for i in all_fields:
    if i not in deprecated_fields and i not in non_required_fields:
        filtered_fields.append(i)
filtered_fields

In [ ]:
# new DF with the filtered fields
covid_df = covid_all[filtered_fields]
covid_df.head() # still think 23 columns is a lot to deal with for the prohect

In [ ]:
# converting date-string format to date
covid_df['date'] = pd.to_datetime(covid_df['date'], format='%Y%m%d')
covid_df.head()

### More Inspection

In [ ]:
covid_df.info()

In [ ]:
# validating results by state and inspecting dates
covid_df.groupby(['state']).agg({'positiveIncrease': 'sum', 'deathIncrease': 'sum', 'date': 'count', 'date': ['min', 'max']})
# https://covid.cdc.gov/covid-data-tracker/#cases_casesinlast7days

In [ ]:
# notice the count of dates is uneven, which is reflected by the start dates variation


In [ ]:
# inspecting viz
covid_viz = covid_df.set_index(['state', 'date']).sort_index()
idx = pd.IndexSlice
select_viz = covid_viz.loc[idx[['MN'], :], :]
select_viz.positiveIncrease

# note: we could slice dates based on variables defined by a slicer on the dashboard

# Data Cleaning
## state_pop_df

In [ ]:
# inspecting columns
state_pop_df.columns

In [ ]:
state_pop_df.info()

In [ ]:
# defining fips field from ID state. This new column will be used for merging
state_pop_df['fips'] = state_pop_df['ID State'].str[-2:]

In [ ]:
state_df = state_pop_df[['State', 'fips', 'Population']]
state_df.head()

# Merging Datasets

In [ ]:
covid_df['fips'].unique()

In [ ]:
covid_wpop = covid_df.merge(state_df, how='inner', on='fips')
covid_wpop.head()

In [ ]:
covid_wpop[covid_wpop.Population == 0]

In [ ]:
covid_wpop.info()

In [ ]:
display(covid_wpop['state'].nunique(), covid_df['state'].nunique())

In [ ]:
display(covid_wpop['state'].unique(), covid_df['state'].unique())

In [ ]:
# checking for omitted states: the result are US territories.
omitted_states = []
for i in covid_df['state'].unique():
    if i not in covid_wpop['state'].unique():
        omitted_states.append(i)
omitted_states

### Seven-day rolling average of new cases, by number of days since X average daily cases first recorded

In [ ]:
covid_indexed = covid_wpop.set_index(['state']).sort_index().sort_values(['state', 'date'])

In [ ]:
covid_rollavg = covid_indexed[['date', 'positive', 'positiveIncrease', 'death', 'deathIncrease']]

In [ ]:
covid_rollavg['positiveRollingAvg'] = covid_rollavg.groupby(level=0)['positiveIncrease'].rolling(window=7).mean().values
covid_rollavg

In [ ]:
covid_rollavg['deathRollingAvg'] = covid_rollavg.groupby(level=0)['deathIncrease'].rolling(window=7).mean().values

In [ ]:
# new cases: looping to get the date >= threshold for each state
state_list = covid_rollavg.index.get_level_values('state').unique()
new_threshold = 10
dates_new = []
for i in state_list:
    new_df = covid_rollavg.loc[i]
    new_df = new_df.loc[new_df['positiveIncrease'] >= new_threshold]
    date = new_df.iloc[0].date
    dates_new.append(date)
dates_new

In [ ]:
new_dict = dict(zip(state_list, dates_new))
new_dict

In [ ]:
covid_rollavg.head()

In [ ]:
covid_rollavg.iloc[0].date

In [ ]:
# deaths: looping to get the date >= threshold for each state
death_threshold = 10
dates_death = []
for i in state_list:
    death_df = covid_rollavg.loc[i]
    death_df = death_df.loc[death_df['death'] >= death_threshold]
    date_d = death_df.iloc[0].date
    dates_death.append(date_d)
dates_death

In [ ]:
death_dict = dict(zip(state_list, dates_death))
death_dict

In [ ]:
covid_rollavg.reset_index(inplace=True)

In [ ]:
covid_rollavg['new_datelapse'] = ''

In [ ]:
covid_rollavg['death_datelapse'] = ''

In [ ]:
# populating new_datelapse with the dete substraction for number of days since reaching the threshold
for x in covid_rollavg.index:
    covid_rollavg.iloc[x, 8] = (covid_rollavg.iloc[x, 1] - new_dict[covid_rollavg.iloc[x, 0]]).days

In [ ]:
# populating death_datelapse with the dete substraction for number of days since reaching the threshold
for x in covid_rollavg.index:
    covid_rollavg.iloc[x, 9] = (covid_rollavg.iloc[x, 1] - death_dict[covid_rollavg.iloc[x, 0]]).days

In [ ]:
covid_rollavg['new_datelapse'] = pd.to_numeric(covid_rollavg['new_datelapse'], errors='coerce')


In [ ]:
covid_rollavg['death_datelapse'] = pd.to_numeric(covid_rollavg['death_datelapse'], errors='coerce')

In [ ]:
covid_rollavg.dtypes

---

## Map Viz DF

In [ ]:
# call the latest available date for each state
# latest_date = covid_wpop.date == covid_wpop.date.max() this works, but what if one of the states is not updated to the latest date?
# need an alternative solution to call the last date, TBD in class
latest_date = covid_wpop.date == covid_wpop.date.max()
map_df = covid_wpop[latest_date]
map_df

In [ ]:
map_df.shape

In [ ]:
map_df.columns

In [ ]:
# reorganizing columns
map_df = map_df[['date', 'State', 'state', 'Population', 
                 'positive', 'positiveIncrease', 'death', 'deathIncrease', 
                 'hospitalizedCurrently', 'hospitalizedIncrease', 'hospitalizedCumulative', 
                 'inIcuCurrently', 'inIcuCumulative', 
                 'onVentilatorCurrently', 'onVentilatorCumulative', 
                 'dataQualityGrade']]
map_df.head()

In [ ]:
map_df['positive_per_100K'] = (map_df['positive'] / (map_df['Population'] / 100000)).round(1)
map_df['death_per_100K'] = (map_df['death'] / (map_df['Population'] / 100000)).round(1)
map_df.head()

In [ ]:
# CASES AND DEATH LAST 7 DAYS?

In [ ]:
##### Emilio's Code Ends Here #####

In [ ]:
#### Allan's Code Starts ####

In [ ]:
#### Allan's Code Ends #### 

In [ ]:
#### Matt's Code Starts ####

In [ ]:
#### Matt's Code Ends #### 